# Mercedes-Benz Greener Manufacturing

In [ ]:
# Step1: Import the required libraries

# linear algebra

In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
# Step2: Read the data from train.csv

In [3]:
df_train=pd.read_csv("F:\\A_Machine_Learning_Projects\\Mercedes_Benz\\train.csv")

In [5]:
# Understanding df_train data

In [4]:
df_train.shape

(4209, 378)

In [5]:
df_train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Step3: Collect the Y values into an array

# seperate the y from the data as we will use this to learn as 
# the prediction output

In [6]:
# seperate the target column from the data
y_train=df_train['y'].values

In [7]:
y_train.shape

(4209,)

In [ ]:
# Step4: Understand the data types we have

# iterate through all the columns which has X in the name of the column

In [9]:
# iterate through all the columns which has X in the name of the column
cols=[c for c in df_train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))
print('Feature types:')
df_train[cols].dtypes.value_counts()

Number of features: 376
Feature types:


int64     368
object      8
dtype: int64

In [ ]:
# Step5: Count the data in each of the columns

In [10]:
counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

In [11]:
print('Constant features: {} Binary features: {} Categorical features: {}\n'.format(*[len(c) for c in counts]))

Constant features: 12 Binary features: 356 Categorical features: 8



In [12]:
print('Constant features:', counts[0])

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [13]:
print('Categorical features:', counts[2])

Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [ ]:
# Step6: Read the test.csv data

In [22]:
df_test=pd.read_csv("F:\\A_Machine_Learning_Projects\\Mercedes_Benz\\test.csv")

In [23]:
df_test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [24]:
# remove columns ID and Y from the data as they are not used for learning
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))
y_train = df_train['y'].values


In [26]:
id_test = df_test['ID'].values

In [27]:
x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

In [45]:
# Step7: Check for null and unique values for test and train sets

In [28]:
def check_missing_values(df):
    if df.isnull().any().any():
        print("There are missing values in the dataframe")
    else:
        print("There are no missing values in the dataframe")
check_missing_values(x_train)
check_missing_values(x_test)

There are no missing values in the dataframe
There are no missing values in the dataframe


In [ ]:
# Step8: If for any column(s), the variance is equal to zero, 
# then you need to remove those variable(s).
# Apply label encoder

In [29]:
for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)

C:\Users\Amit\AppData\Local\Temp/ipykernel_9432/1773519019.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[column] = x_train[column].apply(mapper)
C:\Users\Amit\AppData\Local\Temp/ipykernel_9432/1773519019.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = x_test[column].apply(mapper)


In [30]:
x_train.head()

,X49,X139,X46,X304,X27,X225,X116,X119,X169,X64,...,X359,X369,X132,X90,X152,X176,X155,X83,X310,X150
0,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,1,1,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
3,0,0,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0,0,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [ ]:
# Step9: Make sure the data is now changed into numericals

In [32]:
print('Feature types:')
x_train[cols].dtypes.value_counts()

Feature types:


int64    376
dtype: int64

In [ ]:
# Step10: Perform dimensionality reduction
# Linear dimensionality reduction using Singular Value Decomposition of 
# the data to project it to a lower dimensional space.

In [33]:
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

In [ ]:
# Step11: Training using xgboost

In [38]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [39]:
x_train, x_valid, y_train, y_valid = train_test_split(
        pca2_results_train, 
        y_train, test_size=0.2, 
        random_state=4242)

In [40]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(pca2_results_test)

In [41]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

In [42]:
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[12:38:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.14835	train-r2:-58.35295	valid-rmse:98.26297	valid-r2:-67.63754
[10]	train-rmse:81.27651	train-r2:-38.88428	valid-rmse:80.36433	valid-r2:-44.91014
[20]	train-rmse:66.71610	train-r2:-25.87403	valid-rmse:65.77334	valid-r2:-29.75260
[30]	train-rmse:54.86956	train-r2:-17.17752	valid-rmse:53.88973	valid-r2:-19.64401
[40]	train-rmse:45.24491	train-r2:-11.35979	valid-rmse:44.21971	valid-r2:-12.89996
[50]	train-rmse:37.44729	train-r2:-7.46666	valid-rmse:36.37237	valid-r2:-8.40428
[60]	train-rmse:31.14748	train-r2:-4.85757	valid-rmse:30.01874	valid-r2:-5.40570
[70]	train-rmse:26.08660	train-r2:-3.10872	valid-rmse:24.90890	valid-r2:-3.41053
[80]	train-rmse:22.04638	train-r2:-1.93458	valid-rmse:20.83274	valid-r2:-2.08514
[90]	train-rmse:18.84403	train-r2:-1.14397	valid-rmse:17.60284	valid-r2:-1.20266
[100

In [ ]:
# Step12: Predict your test_df values using xgboost

In [43]:
p_test = clf.predict(d_test)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)

In [44]:
sub.head()

,ID,y
0,1,82.619469
1,2,97.577888
2,3,83.218880
3,4,77.146393
4,5,112.477676
